In [ ]:
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import numpy as np

# Configurații pentru scenarii [cite: 29]
scenarios = [
    {'Y': 0, 'theta': 0.2},
    {'Y': 5, 'theta': 0.5},
    {'Y': 10, 'theta': 0.5} 
]

# a) Calculul distribuției posterioare pentru n
models = {}
traces = {}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, scen in enumerate(scenarios):
    Y_obs = scen['Y']
    theta_val = scen['theta']
    
    with pm.Model() as model:
        # Prior pentru n: Poisson(10) [cite: 28]
        n = pm.Poisson('n', mu=10)
        
        # Likelihood: Binomial(n, theta) [cite: 27]
        # Observăm Y. PyMC va gestiona log-probabilitatea.
        y = pm.Binomial('y', n=n, p=theta_val, observed=Y_obs)
        
        # Sampling
        trace = pm.sample(2000, return_inferencedata=True, progressbar=True)
        traces[f"Y={Y_obs}, theta={theta_val}"] = trace
        
    # Vizualizare az.plot_posterior [cite: 29]
    az.plot_posterior(trace, var_names=['n'], ax=axes[i])
    axes[i].set_title(f"Posterior n | Y={Y_obs}, theta={theta_val}")

plt.tight_layout()
plt.show()

# c) Posterior Predictive Distribution pentru un viitor Y* [cite: 32]
fig2, axes2 = plt.subplots(1, 3, figsize=(18, 5))

for i, (key, trace) in enumerate(traces.items()):
    Y_obs = scenarios[i]['Y']
    theta_val = scenarios[i]['theta']
    
    with pm.Model() as model:
        n = pm.Poisson('n', mu=10)
        y = pm.Binomial('y', n=n, p=theta_val, observed=Y_obs)
        
        # Variabila pentru predicție viitoare Y*
        y_future = pm.Binomial('y_future', n=n, p=theta_val)
        
        # Posterior predictive check
        ppc = pm.sample_posterior_predictive(trace, var_names=['y_future'], progressbar=True)
        
    az.plot_dist(ppc.posterior_predictive['y_future'], ax=axes2[i], color='orange')
    axes2[i].set_title(f"Predictive Y* | {key}")

plt.tight_layout()
plt.show()